In [9]:
# ! pip install langchain==0.3.0 langchain-core==0.3.0 langchain-community==0.3.0

In [10]:
!# pip uninstall -y langchain langchain-core langchain-community

In [11]:
#! pip install langchain langchain-core langchain-community langgraph

In [12]:
import datasets
from langchain_core.documents import Document

# Load the dataset
guest_dataset = datasets.load_dataset(
    "agents-course/unit3-invitees", 
    split="train"
)

docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}",
        ]),
        metadata={
            "name": guest_dataset['name'][i],
            "relation": guest_dataset['relation'][i],
            "description": guest_dataset['description'][i],
            "email": guest_dataset['email'][i]
        }
    )
    for i in range(len(guest_dataset))
]

print(docs[0].page_content)

Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com


In [13]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query) 
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation. Use this tool ONLY when asked about a specific guest, their relation, or their details."
)

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

hf_token= os.getenv("HF_TOKEN")

In [15]:
#! pip install langchain-google-genai

In [16]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, StateGraph

from langchain_google_genai import ChatGoogleGenerativeAI


load_dotenv()
if os.getenv("GEMINI_PAID_KEY"):
    os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_PAID_KEY")
    print("Using GEMINI_PAID_KEY.")
else:
    print("WARNING: GOOGLE_API_KEY environment variable is not set!")


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0
)

# Bind the tool to the LLM
tools = [guest_info_tool]
chat_with_tools = llm.bind_tools(tools)


# Define the Agent State
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

# Build the LangGraph
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

# --- 3. Invoke and Output ---

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("\nAlfred's Response:")
print(response['messages'][-1].content)

Using GEMINI_PAID_KEY.

Alfred's Response:
Lady Ada Lovelace, my best friend, is an esteemed mathematician and is renowned for her pioneering work in mathematics and computing. She is often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example.com.
